In [1]:
import qiskit
from qiskit.circuit import QuantumCircuit
from qiskit.providers.aer.noise import NoiseModel
from qiskit_aer import AerSimulator
import numpy as np
import pandas as pd
from mitiq import Executor, MeasurementResult, Observable
from cirq import DensityMatrixSimulator, depolarize
from mitiq.interface import convert_to_mitiq
import mitiq
from mitiq import zne, benchmarks, pec
from mitiq.pec.representations.depolarizing import represent_operations_in_circuit_with_local_depolarizing_noise

import qiskit
from qiskit.circuit import QuantumCircuit
from qiskit.providers.aer.noise import NoiseModel
from qiskit_aer import AerSimulator
import numpy as np
import pandas as pd
from mitiq import Executor, MeasurementResult, Observable
from cirq import DensityMatrixSimulator, depolarize
from mitiq.interface import convert_to_mitiq
import mitiq
from mitiq import zne, benchmarks, pec
from mitiq.pec.representations import find_optimal_representation
from mitiq.pec.representations.depolarizing import represent_operations_in_circuit_with_local_depolarizing_noise
from mitiq.pec.channels import kraus_to_super
from qiskit.circuit.library import RZGate, SXGate, CXGate
from qiskit.circuit.library.generalized_gates import PauliGate
from qiskit.extensions import UnitaryGate
import time
import copy
import pdb



In [2]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Kraus, SuperOp
from qiskit_aer import AerSimulator
from qiskit.tools.visualization import plot_histogram

# Import from Qiskit Aer noise module
from qiskit_aer.noise import (NoiseModel, QuantumError, ReadoutError,
    pauli_error, depolarizing_error, thermal_relaxation_error)


In [15]:

import qiskit_aer.noise as noise

# Error probabilities
prob_1 = 0.001  # 1-qubit gate
prob_2 = 0.01   # 2-qubit gate

# Depolarizing quantum errors
error_1 = noise.depolarizing_error(prob_1, 1)
error_2 = noise.depolarizing_error(prob_2, 2)

# Add errors to noise model
noise_model = noise.NoiseModel()
noise_model.add_all_qubit_quantum_error(error_1, ['rz', '', 'x'])
noise_model.add_all_qubit_quantum_error(error_2, ['cx'])
print(noise_model)
print(noise_model.noise_instructions)

NoiseModel:
  Basis gates: ['cx', 'id', 'rz', 'sx', 'x']
  Instructions with noise: ['', 'rz', 'cx', 'x']
  All-qubits errors: ['rz', '', 'x', 'cx']
['', 'cx', 'rz', 'x']


In [6]:
noise_dict = pd.read_pickle('./NoiseModel/fakecairo.pkl')
# Load the noise model from the dictionary
noise_model = NoiseModel.from_dict(noise_dict)
print(noise_model)

/tmp/ipykernel_74313/1272081019.py:3: DeprecationWarning: from_dict has been deprecated as of qiskit-aer 0.10.0 and will be removed no earlier than 3 months from that release date.
  noise_model = NoiseModel.from_dict(noise_dict)


NoiseModel:
  Basis gates: ['cx', 'id', 'rz', 'sx', 'x']
  Instructions with noise: ['', 'measure', 'sx', 'reset', 'cx', 'x', 'id']
  Qubits with noise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
  Specific qubit errors: [('id', (0,)), ('id', (1,)), ('id', (2,)), ('id', (3,)), ('id', (4,)), ('id', (5,)), ('id', (6,)), ('id', (7,)), ('id', (8,)), ('id', (9,)), ('id', (10,)), ('id', (11,)), ('id', (12,)), ('id', (13,)), ('id', (14,)), ('id', (15,)), ('id', (16,)), ('id', (17,)), ('id', (18,)), ('id', (19,)), ('id', (20,)), ('id', (21,)), ('id', (22,)), ('id', (23,)), ('id', (24,)), ('id', (25,)), ('id', (26,)), ('sx', (0,)), ('sx', (1,)), ('sx', (2,)), ('sx', (3,)), ('sx', (4,)), ('sx', (5,)), ('sx', (6,)), ('sx', (7,)), ('sx', (8,)), ('sx', (9,)), ('sx', (10,)), ('sx', (11,)), ('sx', (12,)), ('sx', (13,)), ('sx', (14,)), ('sx', (15,)), ('sx', (16,)), ('sx', (17,)), ('sx', (18,)), ('sx', (19,)), ('sx', (20,)), ('sx', (21,)), ('sx', 

In [7]:
two_graphs = [] # the map for cx

for noise in noise_dict['errors']:

    if len(noise['gate_qubits'][0])==2:
        two_graphs.append(noise['gate_qubits'][0])

print(two_graphs)

[(6, 7), (7, 6), (12, 13), (13, 12), (7, 10), (10, 7), (8, 11), (11, 8), (3, 2), (2, 3), (4, 7), (7, 4), (9, 8), (8, 9), (17, 18), (18, 17), (1, 0), (0, 1), (15, 12), (12, 15), (1, 4), (4, 1), (20, 19), (19, 20), (26, 25), (25, 26), (21, 23), (23, 21), (21, 18), (18, 21), (16, 14), (14, 16), (13, 14), (14, 13), (24, 25), (25, 24), (11, 14), (14, 11), (24, 23), (23, 24), (10, 12), (12, 10), (22, 19), (19, 22), (5, 8), (8, 5), (22, 25), (25, 22), (3, 5), (5, 3), (16, 19), (19, 16), (18, 15), (15, 18), (1, 2), (2, 1)]


In [8]:
def execute_with_qiskit_noise(
    circuit: QuantumCircuit,
    noise_model: NoiseModel,
    shots=1,
    ideal = False
):
    qc = circuit.copy()
    qc.save_density_matrix()
    basis_gates = noise_model.basis_gates + ["save_density_matrix"]
    if(ideal):
        job = qiskit.execute(
            qc,
            backend=AerSimulator(),
            optimization_level=0,
            shots=shots
        )
    else:
        job = qiskit.execute(
            qc,
            backend=AerSimulator(method = "density_matrix", noise_model = noise_model),
            basis_gates=basis_gates,
            optimization_level=0,
            shots=shots
        )
    rho = job.result().data()["density_matrix"]
    return rho
    

In [13]:
density_matrix_out = []
ideal_out = []
for pair in [0,1]:
    qc = QuantumCircuit(12)
    for i in range(12):
        qc.h(i)
    
    qc.cx(two_graphs[0][0], two_graphs[0][1])
    noisy_value = execute_with_qiskit_noise(qc,noise_model,1,False)
    print(noisy_value[0].shape)
    density_matrix_out.append(noisy_value)
    ideal_value = execute_with_qiskit_noise(qc,noise_model,1,True)
    ideal_out.append(ideal_value)
    
delta_noise = sum(sum(abs(density_matrix_out[0]-density_matrix_out[1])))
delta_ideal = sum(sum(abs(ideal_value[0]-ideal_value[1])))

print(delta_noise)
print(delta_ideal)

/tmp/ipykernel_74313/2883496640.py:10: DeprecationWarning: The return type of saved density matrices has been changed from a `numpy.ndarray` to a `qiskit.quantum_info.DensityMatrix` as of qiskit-aer 0.10. Accessing numpy array attributes is deprecated and will result in an error in a future release. To continue using saved result objects as arrays you can explicitly cast them using  `np.asarray(object)`.
  print(noisy_value[0].shape)


(4096,)
(4096,)


TypeError: bad operand type for abs(): 'DensityMatrix'

In [6]:
noise_level_1 = 0.01
reps_1 = represent_operations_in_circuit_with_local_depolarizing_noise(qc,noise_level_1)
print(f"{len(reps_1)} OperationRepresentation_1 objects produced, assuming {100 * noise_level_1}% depolarizing noise.")

pec_value_1 = pec.execute_with_pec(qc,execute_with_qiskit_noise(qc,noise_model,1,False),representations=reps_1)
print(f"Error without PEC:                     {abs(ideal_value - noisy_value) :.5f}")
print(f"Error with PEC(correct noise level):   {abs(ideal_value - pec_value_1) :.5f}")

24 OperationRepresentation_1 objects produced, assuming 1.0% depolarizing noise.


/tmp/ipykernel_3299780/2825760794.py:26: DeprecationWarning: The return type of saved density matrices has been changed from a `numpy.ndarray` to a `qiskit.quantum_info.DensityMatrix` as of qiskit-aer 0.10. Accessing numpy array attributes is deprecated and will result in an error in a future release. To continue using saved result objects as arrays you can explicitly cast them using  `np.asarray(object)`.
  return rho[0,0].real


TypeError: unsupported callable

In [4]:
from mitiq.pec.representations import local_depolarizing_kraus
from mitiq.pec.channels import kraus_to_super
import qiskit
from qiskit.circuit import QuantumCircuit
from qiskit.providers.aer.noise import NoiseModel
from qiskit_aer import AerSimulator
import numpy as np
import pandas as pd
from mitiq import Executor, MeasurementResult, Observable
from cirq import DensityMatrixSimulator, depolarize
from mitiq.interface import convert_to_mitiq
import mitiq
from mitiq import zne, benchmarks, pec
from mitiq.pec.representations.depolarizing import represent_operations_in_circuit_with_local_depolarizing_noise


basis_circuit_h = qiskit.QuantumCircuit(1)
basis_circuit_h.h(0)

basis_circuit_hx = qiskit.QuantumCircuit(1)
basis_circuit_hx.h(0)
basis_circuit_hx.x(0)

basis_circuit_hy = qiskit.QuantumCircuit(1)
basis_circuit_hy.h(0)
basis_circuit_hy.y(0)

basis_circuit_hz = qiskit.QuantumCircuit(1)
basis_circuit_hz.h(0)
basis_circuit_hz.z(0)

basis_circuits = [basis_circuit_h, basis_circuit_hx, basis_circuit_hy, basis_circuit_hz] 

for c in basis_circuits:
    print(c)
    
    
# Compute depolarizing superoperator
BASE_NOISE = 0.2
depo_super = kraus_to_super(local_depolarizing_kraus(BASE_NOISE, num_qubits=1))

# Define the superoperator matrix of each noisy operation
super_matrices = [
    depo_super @ kraus_to_super([qiskit.quantum_info.Operator(c).data]) 
    for c in basis_circuits
] 

# Define NoisyOperation objects combining circuits with channel matrices
noisy_operations = [
    pec.NoisyOperation(circuit=c, channel_matrix=m)
    for c, m in zip(basis_circuits, super_matrices)
]

print(f"{len(noisy_operations)} NoisyOperation objects defined.")
print(noisy_operations)
print(super_matrices)

   ┌───┐
q: ┤ H ├
   └───┘
   ┌───┐┌───┐
q: ┤ H ├┤ X ├
   └───┘└───┘
   ┌───┐┌───┐
q: ┤ H ├┤ Y ├
   └───┘└───┘
   ┌───┐┌───┐
q: ┤ H ├┤ Z ├
   └───┘└───┘
4 NoisyOperation objects defined.
[<mitiq.pec.types.types.NoisyOperation object at 0x7f4f5026f9a0>, <mitiq.pec.types.types.NoisyOperation object at 0x7f4e658bae30>, <mitiq.pec.types.types.NoisyOperation object at 0x7f4e67937a30>, <mitiq.pec.types.types.NoisyOperation object at 0x7f4e6588e3b0>]
[array([[ 0.5       +0.j,  0.36666667+0.j,  0.36666667+0.j,
         0.5       +0.j],
       [ 0.36666667+0.j, -0.36666667+0.j,  0.36666667+0.j,
        -0.36666667+0.j],
       [ 0.36666667+0.j,  0.36666667+0.j, -0.36666667+0.j,
        -0.36666667+0.j],
       [ 0.5       +0.j, -0.36666667+0.j, -0.36666667+0.j,
         0.5       +0.j]]), array([[ 0.5       +0.j, -0.36666667+0.j, -0.36666667+0.j,
         0.5       +0.j],
       [ 0.36666667+0.j,  0.36666667+0.j, -0.36666667+0.j,
        -0.36666667+0.j],
       [ 0.36666667+0.j, -0.36666667+0.

In [3]:
import numpy as np
def standard_gate_unitary(name):
    # To be removed with from_dict
    unitary_matrices = {
        ("id", "I"): np.eye(2, dtype=complex),
        ("x", "X"): np.array([[0, 1], [1, 0]], dtype=complex),
        ("y", "Y"): np.array([[0, -1j], [1j, 0]], dtype=complex),
        ("z", "Z"): np.array([[1, 0], [0, -1]], dtype=complex),
        ("h", "H"): np.array([[1, 1], [1, -1]], dtype=complex) / np.sqrt(2),
        ("s", "S"): np.array([[1, 0], [0, 1j]], dtype=complex),
        ("sdg", "Sdg"): np.array([[1, 0], [0, -1j]], dtype=complex),
        ("t", "T"): np.array([[1, 0], [0, np.exp(1j * np.pi / 4)]], dtype=complex),
        ("tdg", "Tdg"): np.array([[1, 0], [0, np.exp(-1j * np.pi / 4)]], dtype=complex),
        ("cx", "CX", "cx_01"): np.array(
            [[1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 1, 0, 0]], dtype=complex
        ),
        ("cx_10",): np.array(
            [[1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 1, 0, 0]], dtype=complex
        ),
        ("cz", "CZ"): np.array(
            [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, -1]], dtype=complex
        ),
        ("swap", "SWAP"): np.array(
            [[1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1]], dtype=complex
        ),
        ("ccx", "CCX", "ccx_012", "ccx_102"): np.array(
            [
                [1, 0, 0, 0, 0, 0, 0, 0],
                [0, 1, 0, 0, 0, 0, 0, 0],
                [0, 0, 1, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 1],
                [0, 0, 0, 0, 1, 0, 0, 0],
                [0, 0, 0, 0, 0, 1, 0, 0],
                [0, 0, 0, 0, 0, 0, 1, 0],
                [0, 0, 0, 1, 0, 0, 0, 0],
            ],
            dtype=complex,
        ),
        ("ccx_021", "ccx_201"): np.array(
            [
                [1, 0, 0, 0, 0, 0, 0, 0],
                [0, 1, 0, 0, 0, 0, 0, 0],
                [0, 0, 1, 0, 0, 0, 0, 0],
                [0, 0, 0, 1, 0, 0, 0, 0],
                [0, 0, 0, 0, 1, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 1],
                [0, 0, 0, 0, 0, 0, 1, 0],
                [0, 0, 0, 0, 0, 1, 0, 0],
            ],
            dtype=complex,
        ),
        ("ccx_120", "ccx_210"): np.array(
            [
                [1, 0, 0, 0, 0, 0, 0, 0],
                [0, 1, 0, 0, 0, 0, 0, 0],
                [0, 0, 1, 0, 0, 0, 0, 0],
                [0, 0, 0, 1, 0, 0, 0, 0],
                [0, 0, 0, 0, 1, 0, 0, 0],
                [0, 0, 0, 0, 0, 1, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 1],
                [0, 0, 0, 0, 0, 0, 1, 0],
            ],
            dtype=complex,
        ),
    }

    return next((value for key, value in unitary_matrices.items() if name in key), None)

In [11]:
a = []
a.append(standard_gate_unitary('abc')) if (standard_gate_unitary('abc') != None)
a.append('1')
a
a[0]==None

True

In [4]:
from qiskit.circuit.library import RZGate

print(RZGate(0.1).to_matrix())

[[0.99875026-0.04997917j 0.        +0.j        ]
 [0.        +0.j         0.99875026+0.04997917j]]


In [1]:
from qiskit.circuit import QuantumCircuit
qc = QuantumCircuit(1)
qc.x(0)
qc.reset(0)
qc.y(0)
qc.draw()

┌───┐     ┌───┐
q: ┤ X ├─|0>─┤ Y ├
   └───┘     └───┘

In [5]:
from qiskit.quantum_info import Operator
from qiskit.compiler import transpile

qc = QuantumCircuit(7)
qc.id(0)
qc.x(1)
# qc.y(2)
# qc.z(3)
qc.rx(np.pi/5,4)
qc.ry(np.pi*8/7,5)
qc.rz(-np.pi,6)
qc.sx(6)
# qc.z(6)

result = transpile(qc, basis_gates=['id', 'x', 'rz', 'sx'])
result.draw()

global phase: π
                                                    
q_0: ───────────────────────────────────────────────
        ┌───┐                                       
q_1: ───┤ X ├───────────────────────────────────────
        └───┘                                       
q_2: ───────────────────────────────────────────────
                                                    
q_3: ───────────────────────────────────────────────
     ┌─────────┐┌────┐┌──────────┐┌────┐┌──────────┐
q_4: ┤ Rz(π/2) ├┤ √X ├┤ Rz(6π/5) ├┤ √X ├┤ Rz(5π/2) ├
     └┬────────┤├────┤├──────────┤├────┤└──────────┘
q_5: ─┤ Rz(-π) ├┤ √X ├┤ Rz(6π/7) ├┤ √X ├────────────
      ├────────┤├────┤└──────────┘└────┘            
q_6: ─┤ Rz(-π) ├┤ √X ├──────────────────────────────
      └────────┘└────┘

In [ ]:
import pdb
basis_noise_circs =  []
for i in range(3):
    qc = QuantumCircuit(2)
    qc.cx(0,1)
    qc.x(1)
    qc.y(0)
    qc.draw()
    
    basis_noise_circs.append(qc)
    
two_graphs = [[0,1],[1,0],[5,6]]
basis_noise_circs_27 = []

for j in range(len(basis_noise_circs)):
    qc_27 = QuantumCircuit(27)
    qc_27.append(basis_noise_circs[j], [two_graphs[i][0],two_graphs[i][1]])
    basis_noise_circs_27.append(qc_27)

qc_27.draw()




In [ ]:
basis_noise_circs_27[0].draw()


In [16]:
qc = QuantumCircuit(2)
qc.cx(0,1)
qc.x(1)
qc.sx(0)
qc.x(0)
qc.x(0)


qc.draw()

t = []
t.append(qc)
a = QuantumCircuit(27)
a.append(t[0],[5,6])
a.draw()
b = []
b.append(a)
b[0].draw()
b.append(a.decompose())
b[1].draw()

# t[0].draw()

q_0: ─────────────────────
                           
 q_1: ─────────────────────
                           
 q_2: ─────────────────────
                           
 q_3: ─────────────────────
                           
 q_4: ─────────────────────
           ┌────┐┌───┐┌───┐
 q_5: ──■──┤ √X ├┤ X ├┤ X ├
      ┌─┴─┐├───┬┘└───┘└───┘
 q_6: ┤ X ├┤ X ├───────────
      └───┘└───┘           
 q_7: ─────────────────────
                           
 q_8: ─────────────────────
                           
 q_9: ─────────────────────
                           
q_10: ─────────────────────
                           
q_11: ─────────────────────
                           
q_12: ─────────────────────
                           
q_13: ─────────────────────
                           
q_14: ─────────────────────
                           
q_15: ─────────────────────
                           
q_16: ─────────────────────
                           
q_17: ─────────────────────
                           
q_18: ─────────────────────
                           
q_19: ─────────────────────
                           
q_20: ─────────────────────
                           
q_21: ─────────────────────
                           
q_22: ─────────────────────
                           
q_23: ─────────────────────
                           
q_24: ─────────────────────
                           
q_25: ─────────────────────
                           
q_26: ─────────────────────

In [13]:
z,_ = convert_to_mitiq(b[1])
print(z)

q_5: ───@───cirq.circuits.qasm_output.QasmUGate(theta=1.0, phi=0.5, lmda=0.5)───
        │
q_6: ───X───cirq.circuits.qasm_output.QasmUGate(theta=1.0, phi=0.0, lmda=1.0)───


In [26]:
basis_noise_circ_cx_1sx_0sx = QuantumCircuit(2)
basis_noise_circ_cx_1sx_0sx.cx(0, 1)
basis_noise_circ_cx_1sx_0sx.sx(0)
basis_noise_circ_cx_1sx_0sx.sx(1)
basis_noise_circ_cx_1sx_0sx.draw()


┌────┐
q_0: ──■──┤ √X ├
     ┌─┴─┐├────┤
q_1: ┤ X ├┤ √X ├
     └───┘└────┘

In [29]:

qc = QuantumCircuit(2)
qc.cx(0,1)
a = kraus_to_super([CXGate().to_matrix()])
print(a)
print(a.shape)

[[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j

In [ ]:
from mitiq.pec.representations.learning import (
    learn_depolarizing_noise_parameter,
)


In [14]:
a = np.array([[[-0.99993277+0.j,  0.        +0.j],[ 0.        +0.j, -0.99838319+0.j]],[[-0.01159577+0.j,  0.        +0.j],[ 0.        +0.j,  0.01161377+0.j]],[[0.        +0.j, 0.05564284+0.j],[0.        +0.j, 0.        +0.j]]])
b = np.array([[[-0.99975178+0.j,  0.        +0.j],[ 0.        +0.j, -0.99797031+0.j]],[[-0.02227953+0.j,  0.        +0.j],[ 0.        +0.j,  0.0223193 +0.j]],[[0.        +0.j, 0.05964149+0.j],[0.        +0.j, 0.        +0.j]]])

a1 = kraus_to_super(a)
b1 = kraus_to_super(b)
d = b1-a1
print(d)

[[-7.33157657e-09+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   4.60981686e-04+0.j]
 [ 0.00000000e+00+0.j -9.56067797e-04+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j -9.56067797e-04+0.j
   0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
  -4.60982934e-04+0.j]]


In [1]:
a = (0,1)
len(a)

2